## Product Tagging 

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt

# https://moonbooks.org/Articles/How-to-sort-a-list-by-the-number-of-occurrences-in-python-/
from collections import Counter

In [ ]:
df = pd.read_csv(('/Users/wolfsinem/product-tagging/data/flipkart_com-ecommerce_sample.csv'))

### All the columns in the dataset

In [ ]:
df.columns

---

#### Since we only need the columns product_name, product_category_tree, description, brand and product_specifications, because these seem interesting, we will only select those for the new dataframe

In [ ]:
new_df = df[['product_name','product_category_tree','description','brand','product_specifications']]

In [ ]:
new_df

### Select one description to test manually

In [ ]:
test_string = new_df['description'][0]
test_string

In [ ]:
# https://stackoverflow.com/questions/2661778/tag-generation-from-a-text-content 
BAD_CHARS = ".!?,\'\""
words = [ word.strip(BAD_CHARS) for word in test_string.strip().split() if len(word) > 4 ]

In [ ]:
words

#### As you can see below, the word 'Shorts' occurs the most, following 'Solid' and 'Women's' 
You could make a new column named 'tags' and append these words to the new column

In [ ]:
count_terms = Counter(words).most_common() # fill e.g. (3) for most common 3 terms
count_terms

### Trying other products

In [ ]:
test_string_2 = new_df['description'][1]
test_string_2

In [ ]:
words_2 = [ word.strip(BAD_CHARS) for word in test_string_2.strip().split() if len(word) > 4 ]
count_terms_2 = Counter(words_2).most_common()
count_terms_2

In [ ]:
test_string_3 = new_df['description'][2]
test_string_3

In [ ]:
words_3 = [ word.strip(BAD_CHARS) for word in test_string_3.strip().split() if len(word) > 4 ]
count_terms_3 = Counter(words_3).most_common()
count_terms_3

In [ ]:
test_string_4 = new_df['description'][4]
test_string_4

In [ ]:
words_4 = [ word.strip(BAD_CHARS) for word in test_string_4.strip().split() if len(word) > 4 ]
count_terms_4 = Counter(words_4).most_common()
count_terms_4

### Per term, count the number of product_descriptions, contain that word. This is the document frequency df(t). 

- search the word 'shorts' in any other description

In [ ]:
list_with_strings = [test_string,test_string_2,test_string_3,test_string_4]

In [ ]:
count_terms[0][0]

In [ ]:
count_terms[0][1]

In [ ]:
def contains_word(s, w):
    return f' {w} ' in f' {s} '

In [ ]:
for i in list_with_strings:
    print(contains_word(i,count_terms[0][0])) # if boolean returns more than 1 true, it means the word occurs in other descriptions too

### The TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
docs = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?']

In [ ]:
cv = CountVectorizer()
# vectorizer = TfidfVectorizer()

word_count_vector = cv.fit_transform(docs)

print(cv.get_feature_names())
print(word_count_vector.shape) #(sentences/unique words)

In [ ]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

In [ ]:
df_idf = pd.DataFrame(tfidf_transformer.idf_,index=cv.get_feature_names(),columns=["idf_weights"])

In [ ]:
df_idf.sort_values(by=['idf_weights']) # the lower the IDF value of a word, the less unique it is

In [ ]:
count_vector = cv.transform(docs)

In [ ]:
tf_idf_vector = tfidf_transformer.transform(count_vector)

In [ ]:
feature_names = cv.get_feature_names()

first_document_vector = tf_idf_vector[0]

df_ = pd.DataFrame(first_document_vector.T.todense(),index=feature_names,columns=['tfidf'])
df_.sort_values(by=['tfidf'],ascending=True)

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.pipeline import Pipeline

# corpus = ['this is the first document',
#            'this document is the second document',
#            'and this is the third one',
#            'is this the first document']


# vocabulary = ['this', 'document', 'first', 'is', 'second', 'the',
#                'and', 'one']


# pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
#                   ('tfid', TfidfTransformer())]).fit(corpus)


# pipe['count'].transform(corpus).toarray()
# # array([[1, 1, 1, 1, 0, 1, 0, 0],
# #        [1, 2, 0, 1, 1, 1, 0, 0],
# #        [1, 0, 0, 1, 0, 1, 1, 1],
# #        [1, 1, 1, 1, 0, 1, 0, 0]])
# pipe['tfid'].idf_
# # array([1.        , 1.22314355, 1.51082562, 1.        , 1.91629073,
# #        1.        , 1.91629073, 1.91629073])
# pipe.transform(corpus).shape
# (4, 8)

---

In [ ]:
description_set = df['description'].drop_duplicates()

In [ ]:
description_set[0]

In [ ]:
new_df

In [ ]:
string_1 = [description_set[0],description_set[1],description_set[2]]
vec_string_1 = cv.fit_transform(string_1)

In [ ]:
print(cv.get_feature_names())
print(vec_string_1.shape)

In [ ]:
tfidf_transformer_1 = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer_1.fit(vec_string_1)

In [ ]:
df_idf_1 = pd.DataFrame(tfidf_transformer_1.idf_,index=cv.get_feature_names(),columns=["idf_weights"])
df_idf_1.sort_values(by=['idf_weights'])

In [ ]:
df_idf_1.head(30)